# Embeddings con LSTM



In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from collections import Counter
import random
from itertools import chain
from pprint import pprint
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

In [2]:
cd ..

c:\Users\Usuario.000\Documents\Facultad\Git\2020-2\APIT-2020-2\ProyectoFinal


In [3]:
from helpers.boletines import get_sentences

In [4]:
cd Embeddings/

c:\Users\Usuario.000\Documents\Facultad\Git\2020-2\APIT-2020-2\ProyectoFinal\Embeddings


## Definición de constantes

In [5]:
# Número de GPUs disponibles. Usar 0 para modo CPU.
ngpu = 1

# Semilla a usar en los generadores de números aleatorios
SEED = 42069
# SEED = random.randint(1, 10000) # En caso de requerir más resultados
random.seed(SEED)
torch.manual_seed(SEED)

print("Random Seed: ", SEED)

Random Seed:  42069


Selecciono el tipo de dispositivo a utilizar (gpu o cpu)

In [6]:
# Decide si queremos correr en gpu o cpu
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")
device

device(type='cuda', index=0)

## Obtener corpus

In [7]:
uri = open('./../mongo_uri.txt', 'r', encoding='utf-8').read()

In [8]:
sentences = get_sentences(uri)

AMLO
1. Asegura AMLO que en campaña todo será amor y paz , que los otros candidatos se ahorren sus provocaciones .
377. “ Y tiene que ver con la alianza de el PRI y de el PAN , no hay que olvidar que los de el PRI , los de el PAN , los de el PRIAN se pusieron de acuerdo para nombrar a los consejeros de el INE , que tampoco son blancas palomas y también se pusieron de acuerdo para nombrar a los magistrados de el Tribunal_Electoral , yo lo denuncié en su momento ” , comentó .
753. “ Antes en las elecciones pasadas no podíamos defender nos frente_a los ataques a la guerra sucia , a las calumnias , porque nos cercaban , nos daban espacios o actuaban de manera tendenciosa en los medios convencionales , pero en esta elección la diferencia la están marcando las redes sociales , ya no pueden ” , expresó .
1129. En otro orden de ideas , informó que si hoy fuesen las elecciones ganaría con 25 puntos de ventaja , es bastante el número de mexicanos que lo apoyan , pero hay quienes crearán de maner

Desenvuelvo los grupos de enunciados en un único arreglo con todos los enunciados de todos los candidatos.

In [9]:
corpus = list(chain(*sentences.values()))

Paso los tokens a minúsculas para reducir el tamaño del vocabulario

In [10]:
corpus = [[w.lower() for w in sent] for sent in corpus]
print(f'El corpus consta de {len(corpus)} enunciados.')

El corpus consta de 6889 enunciados.


Selecciono k oraciones de prueba para el modelo

In [11]:
k = 6889
sentences = random.sample(corpus, k=k)
print('\n'.join([' '.join(sent) for sent in sentences[::k//10]]))

y se comprometió a regresar como presidente electo para traer el plan de desarrollo para uruapan para decir cuánto se invertirá en la región y regresará cada seis meses para evaluar el plan .
previamente , lópez_obrador denunció que están haciendo llamadas telefónicas a_el_por_mayor a ciudadanos para difamar lo y describió que las llamadas provienen de número telefónicos de el país y de el extranjero , es un bombardeo de llamadas .
indicó que la zona franca se mantuvo en toda la línea fronteriza en la época de porfirio_díaz , después en la revolución_mexicana , hasta que llegó carlos_salinas_de_gortari que acabó con el apoyo en la frontera .
lamentó la violencia que vive el país y confió en que a_pesar_de la situación , la gente saldrá a votar , porque tiene mucho interés en participar el 1_de_julio .
a su salida , entrevistado por los medios de comunicación , reiteró su llamado a el voto útil , invitando a las y los ciudadanos a participar libremente en el proyecto de coalición , y ce

## Indexar Corpus

Cuento los tokens en todas las oraciones e imprimo los 40 más frecuentes.

In [13]:
freq = sum([Counter(sent) for sent in sentences], Counter())

In [34]:
n_tipos = len(freq.keys())
print(f'Número de tipos: {num_tipos}')
print(freq.most_common()[::n_tipos//25])

NameError: name 'num_tipos' is not defined

Agrego un diccionario para pasar de palabra a índice numérico

In [15]:
w_to_index = {
    w : ix
    for ix, (w, freq) in enumerate(freq.most_common())
    if freq > 1 # No toma en cuenta los hapax
}

In [16]:
vocab_size = len(w_to_index)
vocab_size

7793

In [17]:
BOS = '<BOS>'
EOS = '<EOS>'
UNK = '<UNK>'

ixBOS = vocab_size
ixEOS = vocab_size + 1
ixUNK = vocab_size + 2

In [18]:
w_to_index[BOS] = ixBOS
w_to_index[EOS] = ixEOS
w_to_index[UNK] = ixUNK

Calculo el nuevo tamaño del vocabulario después de agregar 3 tokens

In [19]:
vocab_size = len(w_to_index)
vocab_size

7796

Creo el diccionario inverso, para convertir de índices a palabras

In [20]:
index_to_w = [ w for w, ix in w_to_index.items() ]

Indexo todo el corpus

In [21]:
def w_to_index_unk(w):
    """
    Le asigna el token UNK a palabras que no aparezcan en el corpus
    """
    try:
        return w_to_index[w] 
    except KeyError:
        return ixUNK

In [22]:
sentences_ix = [
    # Le agrego el inicio y fin de caracter a los enunciados
    [ixBOS] + [ w_to_index_unk(w) for w in sent ] + [ixEOS] 
    for sent in sentences
]

X = [ np.asarray(sent[:-1]) for sent in sentences_ix ]
Y = [ np.asarray(sent[1:]) for sent in sentences_ix ] 

print('Ejemplos X')
print(X[:3])
print('Ejemplos Y')
print(Y[:3])

Ejemplos X
[array([7793,    7,   10,  178,    5,  742,   30,   61,  487,   12,  892,
          3,  206,    1,   98,   12, 2316,   12,  128,  983,   10, 1239,
          8,    4,  341,    7,  763,  182,  409,  764,   12, 3203,    3,
        206,    6]), array([7793, 1525,    2,   29, 1998,    4, 2317,    1,    4, 4420,    5,
          3,  546,    7,    1,    3, 5634,    5,    3,  546,    1, 4421,
        179,    0,    2,   11,   17,    1, 5635,    0,  230,    1, 5636,
          6]), array([7793,  154,   17,    4, 2844,    2,   40,    0,    2, 5637,   34,
        455,    2,   36,    5, 1035,    5,  590,   15,    0,   31, 3696,
          6])]
Ejemplos Y
[array([   7,   10,  178,    5,  742,   30,   61,  487,   12,  892,    3,
        206,    1,   98,   12, 2316,   12,  128,  983,   10, 1239,    8,
          4,  341,    7,  763,  182,  409,  764,   12, 3203,    3,  206,
          6, 7794]), array([1525,    2,   29, 1998,    4, 2317,    1,    4, 4420,    5,    3,
        546,    7,    1,    

In [23]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.30)

In [24]:
print(len(X_train))
print(len(X_test))

4822
2067


Mando los vectores de entrada y salida a tensores en gpu

In [25]:
def to_pytorch_tensor(list_of_lists):
    return [
        torch.from_numpy(l).long().to(device)
        for l in list_of_lists
    ]

In [26]:
X_train = to_pytorch_tensor(X_train)
Y_train = to_pytorch_tensor(Y_train)

X_test = to_pytorch_tensor(X_test)
Y_test = to_pytorch_tensor(Y_test)

## Modelo

### 1. Capa de embedding

### 2. Capa oculta

### 3. Capa de salida

Defino las variables para la red neuronal

In [27]:
# Dimensión de entrada (one-hot), tamaño del vocabulario
D_in = vocab_size

# Dimensión de la capa de embedding
D_emb = 2 # 32

# Dimensión de la capa lstm
D_lstm = 4 # 16

# Dimensión de la capa de salida
D_out = D_in

# Épocas de entrenamiento
num_epochs = 10

# Betas para Adam
beta1 = 0.0001
beta2 = 0.99

# Learning rate
lr =  0.01

In [28]:
class Model(nn.Module):
    def __init__(self, ngpu, D_in, D_emb, D_lstm, D_out):
        super(Model, self).__init__()
        self.ngpu = ngpu
        self.embedding = nn.Embedding(num_embeddings=D_in, embedding_dim=D_emb)#, padding_idx=0)
        self.lstm = nn.LSTM(input_size=D_emb, hidden_size=D_lstm, bias=True)#, batch_first=True)
        # self.out_layer = nn.Sequential(
        #     nn.Embedding(num_embeddings=D_in, embedding_dim=D_emb, padding_idx=0),
        #     nn.LSTM(input_size=D_emb, hidden_size=D_lstm, bias=True, batch_first=True),
        #     nn.Linear(in_features=D_lstm, out_features=D_out, bias=True),
        #     nn.Softmax(dim=D_out)
        # )
        self.linear = nn.Linear(in_features=D_lstm, out_features=D_out, bias=True)
        # self.out_layer = nn.Softmax(dim=1)
        # self.out_layer = nn.Sequential(
        #     nn.Linear(in_features=D_lstm, out_features=D_out, bias=True),
        #     nn.Softmax(dim=1)
        # )

    def forward(self, sentence):
        T = len(sentence)
        # print(sentence)
        # print(T)

        embeddings = self.embedding(sentence).view(T, 1, -1)
        # print('Embeddings')
        # print(embeddings)

        lstm_out, (ht, ct) = self.lstm(embeddings)
        lstm_out = lstm_out.view(T, -1)
        # print('LSTM_out')
        # print(lstm_out)

        preact_out = self.linear(lstm_out).view(T, -1)
        # print('Preact out')
        # print(preact_out)

        return F.log_softmax(preact_out, dim=1)
        # return self.out_layer(preact_out)

In [29]:
model = Model(ngpu, D_in, D_emb, D_lstm, D_out).to(device)
model

Model(
  (embedding): Embedding(7796, 2)
  (lstm): LSTM(2, 4)
  (linear): Linear(in_features=4, out_features=7796, bias=True)
)

In [30]:
# Inicialización de pesos
# def weights_init(m):
#     classname = m.__class__.__name__
#     if classname.find('Embedding') != -1:
#         # Regularizo los pesos
#         n = m.num_embeddings
#         y = 1.0/np.sqrt(n)
#         m.weight.data.uniform_(-y, y)
#     elif classname.find('Linear') != -1:
#         n = m.in_features
#         y = 1.0/np.sqrt(n)
#         m.weight.data.uniform_(-y, y)
#         m.bias.data.fill_(0)
#     elif classname.find('LSTM') != -1:
#         n = m.input_size
#         y = 1.0/np.sqrt(n)
#         m.weight_ih_l0.data.uniform_(-y, y)
#         m.weight_hh_l0.data.uniform_(-y, y)
#         m.bias_ih_l0.data.fill_(0)
#         m.bias_hh_l0.data.fill_(0)

In [31]:
# model.apply(weights_init)

Entropía cruzada como optmizador y SGD como optimizador

In [32]:
criterion = nn.CrossEntropyLoss()
criterion.to(device)
# criterion = nn.NLLLoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=lr, betas=(beta1, beta2))
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

### Entrenamiento

In [33]:
%%time
# for epoch in tqdm(range(num_epochs)):
for epoch in range(num_epochs):
    epoch_loss = 0
    for x, y in zip(X_train, Y_train):
    # for x, y in tqdm(zip(X_train, Y_train)):
        if len(x) == 0:
            tqdm.write('Sentencia vacía')
            continue
        # Limpiamos gradientes acumulados
        model.zero_grad()

        # Forward
        # print(x)
        pred = model(x)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = criterion(pred, y)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        # break
    # tqdm.write(f'Epoch: {epoch:>12} Loss: {loss}')
    # break
    print(f'Coste después de {epoch+1} épocas: {epoch_loss}')

Coste después de 1 épocas: 39828.01940870285
Coste después de 2 épocas: 33264.7314453125
Coste después de 3 épocas: 31796.366305589676
Coste después de 4 épocas: 31189.253276586533
Coste después de 5 épocas: 30796.07231426239
Coste después de 6 épocas: 30489.32657933235
Coste después de 7 épocas: 30223.72301888466
Coste después de 8 épocas: 29971.639236211777
Coste después de 9 épocas: 29699.824879407883
Coste después de 10 épocas: 29454.79591369629
Wall time: 4min 23s


In [ ]:
# list(model.named_parameters())

In [36]:
def print_ix_sentence(sentence):
    print(' '.join(index_to_w[ix] for ix in sentence.data))

In [37]:
# See what the scores are after training
with torch.no_grad():
    for sentence, y in zip(X_test[:10], Y_test[:10]):
        print('===================================================')
        print('- Probando enunciado:')
        print_ix_sentence(sentence)
        print()

        out = model(sentence)
        prediccion = torch.argmax(out, dim=1)
        print('- Predicción:')
        print_ix_sentence(prediccion)
        print
        print('- Real:')
        print_ix_sentence(y)
        print('\n')

- Probando enunciado:
<BOS> a el dejar en claro que los mexicanos no se quedarán <UNK> de <UNK> , ricardo_anaya advirtió que este <UNK> requiere que el estado_mexicano tome cartas en el asunto para defender los intereses nacionales y los derechos de los mexicanos que viven en los estados_unidos .

- Predicción:
, , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , <EOS>
- Real:
a el dejar en claro que los mexicanos no se quedarán <UNK> de <UNK> , ricardo_anaya advirtió que este <UNK> requiere que el estado_mexicano tome cartas en el asunto para defender los intereses nacionales y los derechos de los mexicanos que viven en los estados_unidos . <EOS>


- Probando enunciado:
<BOS> otra acción que realizará , detalló , es que se terminarán todos los lujos en el gobierno federal , además se <UNK> a el avión presidencial que costó 7_mil_500_millones_de_pesos , se venderá la flotilla de aviones y helicópteros de el gobierno .

- Predicción:
, , , , , , , 

In [40]:
list(model.named_parameters())[0]

('embedding.weight',
 Parameter containing:
 tensor([[ 1.6694, -1.6333],
         [ 0.5349,  0.8884],
         [ 0.8397, -2.0565],
         ...,
         [-1.9192,  0.0346],
         [-0.0697,  1.1478],
         [-0.6999, -0.4746]], device='cuda:0', requires_grad=True))